In [1]:
import torch
import time
from torch import nn, optim
import torch.nn.functional as F
import sys
sys.path.append("..")
from util import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    if first_block:
        assert in_channels == out_channels
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(d2l.Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(d2l.Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

ResNet-18 结构
- 每个 Residual 2 个 conv(Not 1*1)
- 每个 blk 2 个 residual
- 共计 4 个 blk 16 个 conv
- 输入的 conv ＋ 输出的 fc
- 共计 18 层

In [3]:
net = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                    nn.BatchNorm2d(64),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
net.add_module("resnet_block1", resnet_block(64, 64, 2, first_block=True))
net.add_module("resnet_block2", resnet_block(64, 128, 2))
net.add_module("resnet_block3", resnet_block(128, 256, 2))
net.add_module("resnet_block4", resnet_block(256, 512, 2))
net.add_module("global_avg_pool", d2l.GlobalAvgPool2d())
net.add_module("fc", nn.Sequential(d2l.FlattenLayer(), nn.Linear(512, 10)))

In [4]:
X = torch.rand((1, 1, 224, 224))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

0  output shape:	 torch.Size([1, 64, 112, 112])
1  output shape:	 torch.Size([1, 64, 112, 112])
2  output shape:	 torch.Size([1, 64, 112, 112])
3  output shape:	 torch.Size([1, 64, 56, 56])
resnet_block1  output shape:	 torch.Size([1, 64, 56, 56])
resnet_block2  output shape:	 torch.Size([1, 128, 28, 28])
resnet_block3  output shape:	 torch.Size([1, 256, 14, 14])
resnet_block4  output shape:	 torch.Size([1, 512, 7, 7])
global_avg_pool  output shape:	 torch.Size([1, 512, 1, 1])
fc  output shape:	 torch.Size([1, 10])


In [5]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

OK
C:\Users\Administrator\AppData\Roaming\SPB_16.6\Datasets\FashionMNIST\FashionMNIST\processed\training.pt
OK
C:\Users\Administrator\AppData\Roaming\SPB_16.6\Datasets\FashionMNIST\FashionMNIST\processed\training.pt
OK
C:\Users\Administrator\AppData\Roaming\SPB_16.6\Datasets\FashionMNIST\FashionMNIST\processed\training.pt
OK
C:\Users\Administrator\AppData\Roaming\SPB_16.6\Datasets\FashionMNIST\FashionMNIST\processed\training.pt
training on  cuda
epoch 1, loss 0.3800, train acc 0.860, test acc 0.864, time 423.8 sec
epoch 2, loss 0.1253, train acc 0.908, test acc 0.906, time 534.9 sec
epoch 3, loss 0.0706, train acc 0.923, test acc 0.887, time 592.9 sec
epoch 4, loss 0.0460, train acc 0.931, test acc 0.903, time 613.5 sec
epoch 5, loss 0.0320, train acc 0.941, test acc 0.921, time 642.1 sec
